In [181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [182]:
Obesity = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
Obesity.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.00,1.62,64.00,yes,no,2.00,3.00,Sometimes,no,2.00,no,0.00,1.00,no,Public_Transportation,Normal_Weight
1,Female,21.00,1.52,56.00,yes,no,3.00,3.00,Sometimes,yes,3.00,yes,3.00,0.00,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.00,1.80,77.00,yes,no,2.00,3.00,Sometimes,no,2.00,no,2.00,1.00,Frequently,Public_Transportation,Normal_Weight
3,Male,27.00,1.80,87.00,no,no,3.00,3.00,Sometimes,no,2.00,no,2.00,0.00,Frequently,Walking,Overweight_Level_I
4,Male,22.00,1.78,89.80,no,no,2.00,1.00,Sometimes,no,2.00,no,0.00,0.00,Sometimes,Public_Transportation,Overweight_Level_II


In [183]:
#转化变量
Obesity.loc[Obesity['Gender'] == 'Female','Gender'] = 0
Obesity.loc[Obesity['Gender'] == 'Male','Gender'] = 1
Obesity.loc[Obesity['family_history_with_overweight'] == 'yes','family_history_with_overweight'] = 1
Obesity.loc[Obesity['family_history_with_overweight'] == 'no','family_history_with_overweight'] = 0
Obesity.loc[Obesity['FAVC'] == 'yes','FAVC'] = 1
Obesity.loc[Obesity['FAVC'] == 'no','FAVC'] = 0
Obesity.loc[Obesity['SMOKE'] == 'yes','SMOKE'] = 1
Obesity.loc[Obesity['SMOKE'] == 'no','SMOKE'] = 0
Obesity.loc[Obesity['SCC'] == 'yes','SCC'] = 1
Obesity.loc[Obesity['SCC'] == 'no','SCC'] = 0
Obesity.loc[Obesity['NObeyesdad'] == 'Insufficient_Weight','NObeyesdad'] = 1
Obesity.loc[Obesity['NObeyesdad'] == 'Normal_Weight','NObeyesdad'] = 2
Obesity.loc[Obesity['NObeyesdad'] == 'Overweight_Level_I','NObeyesdad'] = 3
Obesity.loc[Obesity['NObeyesdad'] == 'Overweight_Level_II','NObeyesdad'] = 4
Obesity.loc[Obesity['NObeyesdad'] == 'Obesity_Type_I','NObeyesdad'] = 5
Obesity.loc[Obesity['NObeyesdad'] == 'Obesity_Type_II','NObeyesdad'] = 6
Obesity.loc[Obesity['NObeyesdad'] == 'Obesity_Type_III','NObeyesdad'] = 7
Obesity.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.00,1.62,64.00,1,0,2.00,3.00,Sometimes,0,2.00,0,0.00,1.00,no,Public_Transportation,2
1,0,21.00,1.52,56.00,1,0,3.00,3.00,Sometimes,1,3.00,1,3.00,0.00,Sometimes,Public_Transportation,2
2,1,23.00,1.80,77.00,1,0,2.00,3.00,Sometimes,0,2.00,0,2.00,1.00,Frequently,Public_Transportation,2
3,1,27.00,1.80,87.00,0,0,3.00,3.00,Sometimes,0,2.00,0,2.00,0.00,Frequently,Walking,3
4,1,22.00,1.78,89.80,0,0,2.00,1.00,Sometimes,0,2.00,0,0.00,0.00,Sometimes,Public_Transportation,4


In [184]:
#转化哑变量
Obesity_CAEC = pd.get_dummies(Obesity['CAEC'])
Obesity_CAEC.columns = ['CAEC_Always','CAEC_Frequently','CAEC_Sometimes','CAEC_no']
Obesity_CALC = pd.get_dummies(Obesity['CALC'])
Obesity_CALC.columns = ['CALC_Always','CALC_Frequently','CALC_Sometimes','CALC_no']
Obesity_MTRANS = pd.get_dummies(Obesity['MTRANS'])
Obesity_MTRANS.columns = ['MTRANS_Automobile','MTRANS_Bike','MTRANS_Motorbike','MTRANS_Public_Transportation','MTRANS_Walking']

In [185]:
Obesity1 = Obesity.drop(columns=['CAEC','CALC','MTRANS','NObeyesdad'])
Obesity1 = pd.concat([Obesity1,Obesity_CAEC,Obesity_CALC,Obesity_MTRANS,Obesity['NObeyesdad']],axis=1)
Obesity1.drop(axis=1,columns=['CALC_Always'], inplace=True) #'CALC_Always'的标准差过小，剔除
Obesity1.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH2O,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,0,21.00,1.62,64.00,1,0,2.00,3.00,0,2.00,0,0.00,1.00,0,0,1,0,0,0,1,0,0,0,1,0,2
1,0,21.00,1.52,56.00,1,0,3.00,3.00,1,3.00,1,3.00,0.00,0,0,1,0,0,1,0,0,0,0,1,0,2
2,1,23.00,1.80,77.00,1,0,2.00,3.00,0,2.00,0,2.00,1.00,0,0,1,0,1,0,0,0,0,0,1,0,2
3,1,27.00,1.80,87.00,0,0,3.00,3.00,0,2.00,0,2.00,0.00,0,0,1,0,1,0,0,0,0,0,0,1,3
4,1,22.00,1.78,89.80,0,0,2.00,1.00,0,2.00,0,0.00,0.00,0,0,1,0,0,1,0,0,0,0,1,0,4


In [186]:
Obesity_data = np.array(Obesity1)
print(Obesity_data.shape)

(2111, 26)


In [187]:
X = Obesity_data[:,0:-1]
y = Obesity_data[:,-1].astype('int')
print(X.shape)
print(y.shape)

(2111, 25)
(2111,)


In [188]:
Obesity1.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH2O,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,0,21.00,1.62,64.00,1,0,2.00,3.00,0,2.00,0,0.00,1.00,0,0,1,0,0,0,1,0,0,0,1,0,2
1,0,21.00,1.52,56.00,1,0,3.00,3.00,1,3.00,1,3.00,0.00,0,0,1,0,0,1,0,0,0,0,1,0,2
2,1,23.00,1.80,77.00,1,0,2.00,3.00,0,2.00,0,2.00,1.00,0,0,1,0,1,0,0,0,0,0,1,0,2
3,1,27.00,1.80,87.00,0,0,3.00,3.00,0,2.00,0,2.00,0.00,0,0,1,0,1,0,0,0,0,0,0,1,3
4,1,22.00,1.78,89.80,0,0,2.00,1.00,0,2.00,0,0.00,0.00,0,0,1,0,0,1,0,0,0,0,1,0,4


In [189]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [190]:
train_list = ['Gender','Age','Height','Weight','family_history_with_overweight','FAVC','FCVC','NCP','SMOKE','CH20',
              'SCC','FAF','TUE','CAEC_Always','CAEC_Frequently','CAEC_Sometimes','CAEC_no','CALC_Frequently','CALC_Sometimes','CALC_no','MTRANS_Automobile','MTRANS_Bike','MTRANS_Motorbike','MTRANS_Public_Transportation','MTRANS_Walking']
X_tr = pd.DataFrame(X_train,columns = train_list )
X_Lp_train = X_tr.copy()
X_Lp_train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH20,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,1,23.00,1.71,90.50,1,1,2.00,3.00,0,1.53,0,0.97,1.00,0,0,1,0,0,0,1,0,0,0,1,0
1,1,28.38,1.77,113.24,1,1,2.31,3.00,0,2.15,0,0.00,1.58,0,0,1,0,0,1,0,1,0,0,0,0
2,0,19.37,1.63,82.00,1,1,2.81,2.88,0,1.00,0,0.00,1.38,0,0,1,0,0,1,0,0,0,0,1,0
3,1,26.96,1.78,112.96,1,1,2.26,3.00,0,2.09,0,0.00,1.84,0,0,1,0,0,1,0,1,0,0,0,0
4,0,21.01,1.72,131.93,1,1,3.00,3.00,0,1.68,0,1.65,0.86,0,0,1,0,0,1,0,0,0,0,1,0


In [191]:
#标准化训练集
for i in train_list:
    X_tr[i] = (X_tr[i] - np.mean(X_tr[i]))/np.std(X_tr[i])
X_tr.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH20,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.99,-0.21,0.06,0.15,0.47,0.36,-0.80,0.38,-0.14,-0.77,-0.23,-0.04,0.57,-0.17,-0.34,0.44,-0.17,-0.19,-1.43,1.55,-0.52,-0.06,-0.07,0.58,-0.17
1,0.99,0.66,0.70,1.01,0.47,0.36,-0.21,0.38,-0.14,0.24,-0.23,-1.18,1.52,-0.17,-0.34,0.44,-0.17,-0.19,0.70,-0.65,1.92,-0.06,-0.07,-1.73,-0.17
2,-1.01,-0.79,-0.72,-0.18,0.47,0.36,0.71,0.23,-0.14,-1.63,-0.23,-1.18,1.19,-0.17,-0.34,0.44,-0.17,-0.19,0.70,-0.65,-0.52,-0.06,-0.07,0.58,-0.17
3,0.99,0.43,0.85,1.00,0.47,0.36,-0.30,0.38,-0.14,0.14,-0.23,-1.18,1.95,-0.17,-0.34,0.44,-0.17,-0.19,0.70,-0.65,1.92,-0.06,-0.07,-1.73,-0.17
4,-1.01,-0.53,0.24,1.73,0.47,0.36,1.08,0.38,-0.14,-0.52,-0.23,0.75,0.34,-0.17,-0.34,0.44,-0.17,-0.19,0.70,-0.65,-0.52,-0.06,-0.07,0.58,-0.17


In [192]:
#标准化测试集
X_te = pd.DataFrame(X_test,columns = train_list)
X_Lp_test = X_te.copy()
for i in train_list:
    X_te[i] = (X_te[i] - np.mean(X_te[i]))/np.std(X_te[i])
X_te.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH20,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,-1.02,0.23,-1.04,0.63,0.49,0.37,1.12,0.47,-0.15,-1.64,-0.19,-1.08,0.53,-0.13,-0.40,0.46,-0.13,-0.18,0.75,-0.70,-0.54,-0.04,-0.08,0.58,-0.15
1,0.98,-0.94,1.03,0.84,0.49,0.37,-0.75,-0.84,-0.15,1.15,-0.19,-0.03,1.23,-0.13,-0.40,0.46,-0.13,-0.18,-1.34,1.44,-0.54,-0.04,-0.08,0.58,-0.15
2,0.98,0.22,1.32,0.74,0.49,0.37,0.98,0.47,-0.15,0.23,-0.19,0.16,1.37,-0.13,-0.40,0.46,-0.13,-0.18,0.75,-0.70,-0.54,-0.04,-0.08,0.58,-0.15
3,-1.02,-0.62,0.13,1.65,0.49,0.37,1.12,0.47,-0.15,-0.42,-0.19,0.54,0.29,-0.13,-0.40,0.46,-0.13,-0.18,0.75,-0.70,-0.54,-0.04,-0.08,0.58,-0.15
4,0.98,-0.21,0.51,1.29,0.49,0.37,-0.75,0.47,6.56,-0.02,-0.19,1.15,0.53,-0.13,-0.40,0.46,-0.13,-0.18,0.75,-0.70,-0.54,-0.04,-0.08,0.58,-0.15


In [193]:
#把所有的变量合起来形成一个变量矩阵
list_in_1 = []
list_in_2 = []
n = 0
for i in range(25):
    list_in_1.append(np.array(X_tr[train_list[n]]).reshape(-1,1))
    list_in_2.append(np.array(X_te[train_list[n]]).reshape(-1,1))
    n = n + 1
X1_train = np.hstack(tuple(list_in_1))
X1_test = np.hstack(tuple(list_in_2))
X1_train.shape

(1583, 25)

In [194]:
X1_trainval = X1_train
y_trainval = y_train
X1_trainval.shape,y_trainval.shape

((1583, 25), (1583,))

knn

In [195]:
#训练knn模型
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5,metric='euclidean')
knn.fit(X1_train, y_train) 

KNeighborsClassifier(metric='euclidean')

In [196]:
#模型准确率
OA_knn = knn.score(X1_test, y_test) 
print(OA_knn)

0.7916666666666666


In [197]:
#混淆矩阵
from sklearn.metrics import confusion_matrix
y_pred = knn.predict(X1_test) 
CM_knn = confusion_matrix(y_test, y_pred)
print(CM_knn)

[[74  5  0  0  0  0  0]
 [16 30  5  6  1  1  0]
 [ 1  8 45  1 13  1  0]
 [ 1  4  8 56 10  7  0]
 [ 1  3  0  5 67  7  1]
 [ 1  1  0  0  1 67  1]
 [ 0  0  0  0  0  1 79]]


In [198]:
X1_train_pre = X1_train
y_train_pre = y_train

X1_trainval = X1_train
y_trainval = y_train

#分离训练集和验证集
X1_train, X1_val, y_train, y_val = train_test_split(X1_trainval, y_trainval,test_size=0.33)

#寻找验证集的最佳准确率
val_scores = []              
neighbors = np.arange(1, 15, 2)     
for i in neighbors:
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X1_train, y_train)
    val_scores.append(knn.score(X1_val, y_val))
    
print("Best validation accuracy: {:.3f}".format(np.max(val_scores)))     
best_n_neighbors = neighbors[np.argmax(val_scores)]
print("Best number of neighbors: {}".format(best_n_neighbors))
knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)

#测试最佳近邻下的准确率 
knn.fit(X1_trainval, y_trainval)                
print("Test-set accuracy: {:.3f}".format(knn.score(X1_test, y_test)))

X1_train = X1_train_pre
y_train = y_train_pre

Best validation accuracy: 0.788
Best number of neighbors: 3
Test-set accuracy: 0.812


k-fold

In [199]:
#cv=6进行交叉验证
from sklearn.model_selection import cross_val_score
cross_val_scores = []
for i in neighbors:
    knn = KNeighborsClassifier(n_neighbors=i)   
    scores = cross_val_score(knn, X1_trainval, y_trainval, cv=6)    
    cross_val_scores.append(np.mean(scores))
    
print("Best cross-validation accuracy: {:.3f}".format(np.max(cross_val_scores)))
best_n_neighbors = neighbors[np.argmax(cross_val_scores)]
print("Best number of neighbors: {}".format(best_n_neighbors))
knn = KNeighborsClassifier(n_neighbors=best_n_neighbors)
knn.fit(X1_trainval, y_trainval)
print("Test-set score: {:.3f}".format(knn.score(X1_test, y_test))) 

Best cross-validation accuracy: 0.823
Best number of neighbors: 1
Test-set score: 0.831


lasso回归

In [200]:
import numpy as np
import pandas as pd
pd.set_option('max_rows',None)
pd.set_option('max_columns',None)
pd.options.display.float_format = '{:,.2f}'.format

In [201]:
X_Lp_train.head() 

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH20,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,1,23.00,1.71,90.50,1,1,2.00,3.00,0,1.53,0,0.97,1.00,0,0,1,0,0,0,1,0,0,0,1,0
1,1,28.38,1.77,113.24,1,1,2.31,3.00,0,2.15,0,0.00,1.58,0,0,1,0,0,1,0,1,0,0,0,0
2,0,19.37,1.63,82.00,1,1,2.81,2.88,0,1.00,0,0.00,1.38,0,0,1,0,0,1,0,0,0,0,1,0
3,1,26.96,1.78,112.96,1,1,2.26,3.00,0,2.09,0,0.00,1.84,0,0,1,0,0,1,0,1,0,0,0,0
4,0,21.01,1.72,131.93,1,1,3.00,3.00,0,1.68,0,1.65,0.86,0,0,1,0,0,1,0,0,0,0,1,0


In [202]:
X_Lp_test.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH20,SCC,FAF,TUE,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0,26.00,1.61,102.69,1,1,3.00,3.00,0,1.02,0,0.11,1.00,0,0,1,0,0,1,0,0,0,0,1,0
1,1,18.00,1.80,108.32,1,1,2.00,1.94,0,2.71,0,1.00,1.43,0,0,1,0,0,0,1,0,0,0,1,0
2,1,25.92,1.82,105.80,1,1,2.93,3.00,0,2.15,0,1.17,1.51,0,0,1,0,0,1,0,0,0,0,1,0
3,0,20.22,1.72,129.47,1,1,3.00,3.00,0,1.76,0,1.49,0.86,0,0,1,0,0,1,0,0,0,0,1,0
4,1,23.00,1.75,120.00,1,1,2.00,3.00,1,2.00,0,2.00,1.00,0,0,1,0,0,1,0,0,0,0,1,0


In [203]:
#把所有的变量合起来形成一个变量矩阵
list_in_3 = []
list_in_4 = []
n = 0
for i in range(25):
    list_in_3.append(np.array(X_Lp_train[train_list[n]]).reshape(-1,1))
    list_in_4.append(np.array(X_Lp_test[train_list[n]]).reshape(-1,1))
    n = n + 1
X_Lp_train = np.hstack(tuple(list_in_3))
X_Lp_test = np.hstack(tuple(list_in_4))
X_Lp_train.shape

(1583, 25)

In [213]:
#找到模型最优的lambda值
from sklearn.model_selection import GridSearchCV
param_grid = {'C': np.logspace(-3, 0, 13)}
model_lasso = linear_model.LogisticRegression(penalty = 'l1',max_iter = 5000,solver = 'liblinear')
model1 = GridSearchCV(model_lasso,param_grid,cv = 10)
model1.fit(X_Lp_train,y_train)
print(model1.best_params_)            
print("Test-set accuracy: {:.3f}".format(model1.score(X_test, y_test)))

{'C': 1.0}
Test-set accuracy: 0.750


In [214]:
#混淆矩阵
ylr_pred = model1.predict(X_Lp_test)
ylr_pred_prob =  model1.predict_proba(X_Lp_test)
CM_lr = confusion_matrix(y_test, ylr_pred)
print(CM_lr)

[[76  2  1  0  0  0  0]
 [ 9 31  9  8  2  0  0]
 [ 0 10 39  5 15  0  0]
 [ 0  3 13 38 31  1  0]
 [ 0  1  5  8 63  7  0]
 [ 0  0  0  0  1 69  1]
 [ 0  0  0  0  0  0 80]]


In [215]:
#最优模型的系数
lasso = linear_model.Lasso(alpha=1)
lasso.fit(X_test,y_test)
print(lasso.coef_)

[-0.          0.00650747 -0.          0.06824615  0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.          0.         -0.
 -0.          0.         -0.         -0.         -0.          0.
 -0.        ]


从lasso回归结果来看，体重对于肥胖症的影响最为显著。

岭回归

In [114]:
#找到模型最优的lambda值
model_ridge = linear_model.LogisticRegression(penalty = 'l2',max_iter = 50000)
model2 = GridSearchCV(model_ridge,param_grid,cv = 10)
model2.fit(X_test,y_test)
print(model2.best_params_)
print("Test-set accuracy: {:.3f}".format(model2.score(X_test, y_test)))

{'C': 1.0}
Test-set accuracy: 0.826


In [115]:
ylr_pred = model1.predict(X_Lp_test)
ylr_pred_prob =  model1.predict_proba(X_Lp_test)
CM_lr = confusion_matrix(y_test, ylr_pred)
print(CM_lr)

[[10  7  7 10  7 10 14]
 [ 8 17  9 14 15 14 12]
 [ 7 13  9  3 13 10 16]
 [ 8 10 11  7 17 14 12]
 [12 13 12 14  8 10 18]
 [ 6  9  2 10  3 16 13]
 [16  8 13 11 11  9 10]]


In [116]:
ridge = linear_model.Ridge(alpha=1)
ridge.fit(X_test,y_test)
print(ridge.coef_)

[-1.61509065e-01  3.08461225e-02 -4.86545415e+00  7.33715128e-02
  2.59460987e-01  1.27538254e-01 -8.28964874e-03 -2.27864240e-03
 -2.68517773e-01 -2.14237985e-02  6.83700264e-02 -1.20478293e-01
 -1.76840286e-02  5.24903667e-03 -1.96551174e-01  9.75306774e-02
  9.37714597e-02  1.16819502e-01 -5.54708750e-02  5.77885500e-02
 -8.43726905e-02 -1.44431136e-01  3.05791459e-02  1.45772169e-01
  5.24525116e-02]


由岭回归结果来看，家庭超重历史这一因素对于肥胖症具有最强的正作用，身高对于肥胖症具有最强的负作用。